In [1]:
from ecdsa.ellipticcurve import *

# Diffie Hellman Shared Secret
Elliptic Cryptography is based on the [Discrete Logarithm Problem](https://jeremykun.com/2014/03/31/elliptic-curve-diffie-hellman/). In essense, we can simplify the idea to
> Adding  is easy on elliptic curves,  but **undoing** addition seems hard

More formally, we can give the following definition: Let $G$ be an additive  group, and let $x,y$ be elements of $G$ so that $x=ny$ for some integer $n$. The Discrete  Logarithm Problem asks one to find $n$ when given $x$ and $y$.

In integers, this problem is quite easy.
If I have:
- $x=12$
- $y=4185072$
Then, I can compute that $y=41805072=348756*12=348756x \rightarrow  n=348756$.

> Division for integers is efficient, **but for elliptic curves this is not the case**.

Here is a toy problem testing my understanding of DH's shared secret protocol.

In [3]:
# Diffie Hellman Shared Secret POC
def sendDH(privateKey, generator, sendFunction):
   return sendFunction(privateKey * generator)

def receiveDH(privateKey, receiveFunction):
   return privateKey * receiveFunction()

prime = 3851
a = 324
b = 1287
myCurve = CurveFp(prime, a, b)
basePoint = Point(myCurve, 920, 303)

In [4]:
aliceSecretKey = 233    # generateSecretKey(8)
bobSecretKey = 25       # generateSecretKey(8)

alicePublicKey = sendDH(aliceSecretKey, basePoint, lambda x:x)
bobPublicKey = sendDH(bobSecretKey, basePoint, lambda x:x)

sharedSecret1 = receiveDH(bobSecretKey, lambda: alicePublicKey)
sharedSecret2 = receiveDH(aliceSecretKey, lambda: bobPublicKey)

In [6]:
print(myCurve)
print(basePoint)
print('Shared secret is %s == %s' % (sharedSecret1, sharedSecret2))

(920,303)
Shared secret is (1001,3826) == (1001,3826)


# Algorithm Overview
Steps to implement
1. Hashing password into elliptic curve
    - PWD entered by user
    - SHA-256 computation, hashing into the NIST P-256 Curve
    - this is computed on input + iteration_counter $\rightarrow Z_q$.
    - computed value is considered $x$ coord of a point on curve if $y$-value is associated with it is a quadratic residue (i.e. $x,y$ satisfy the curve equation).
    - this is repeated until a curve element is obtained. (which is the output)
    - Password is concatenated with domain name and input into H'. (ADD RESISTANCE AGAINST PHISHING)
2. FK-PTR OPRF protocol
    1. EXTENSION:
        - Blind the password with OPRF
            - OPRF: $F_k(x)=H(x,(H'(x))^k)$
                - input $x$ from client
                - $k$ is from device
                - H maps from arbitrary length string $\rightarrow e \in \{0,1\}^\tau$, $\tau$ is a security parameter
                - H' maps from arbitrary length string $\rightarrow g \in G$
                    - H' is the "Hash into Elliptic Curve" function, which maps the password into a point on NIST P-256 curve
            - works over a group $G$ of prime order $p$ (e.g. NIST P-256 group)
        - extension picks a random number $\rho \in Z_q$ and raises the hash value of the input to the power $\rho$.
            - this blinding factor $\rho$ hides the password with information-theoretic security)
        SEND THIS AS $\alpha$
    2. DEVICE
        - check if $\alpha$ is $\in G$
        - compute and SEND BACK $\beta = \alpha^k$
    3. BACK TO EXTENSION
        - check if $\beta$ is $\in G$
        - raise the recieved value to the power of $\rho^{-1} \in Z_q$
        - then compute the SHA-256 hash of calculated value.
    4. BACK TO RWD PASSWORD
        - (same as PwdHash implementation)
        - encoded to a random combination of letters, numbers and symbols matching the passwrod requirement of the visited website and entered into pwd field of login page.
        
## N.B.
- taking the exponential in a group is just repetition of the operation
    - i.e. $\forall x \in G, n \in Z, x^n = \underbrace{x+...+x}_\text{n times}$.
    - see this [crypto stack exchange thread](https://crypto.stackexchange.com/questions/57768/exponentiation-in-ecc)
- how to take the inverse power in a group
    - Raise the point to the power $a^{-1} \in Z_p$
    - this is fairly easy to do with [euclidean algorithm](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse)
    - see [stack exchange link](https://math.stackexchange.com/questions/471269/point-division-in-elliptic-curve-cryptography) for more info
- Instantiation assumes a cyclic group $G$ of prime order $q$, $|q|=\tau$, with generator $g$. 
    - At init, User chooses master password pwd, while Device chooses and stores $k \leftarrow Z_q$.
- H which looks like it accepts two arguments when it's called in $H(x, (H'(x))^k)$ really just means hash it all at once, by appending it.
    - in the paper, it describes this step as "Client hashes this value $(H'(pwd|domain))^k$ with the pwd to obtain rwd"
    - in the implementation, they use `crypto_generichash_update()` from [source.](https://github.com/stef/libsphinx/blob/master/src/sphinx.c)
    - furthermore, in the documentation for `<sodium.h>` they use this function to compute the hash on a multi-part [example](https://libsodium.gitbook.io/doc/hashing/generic_hashing#multi-part-example-with-a-key)
    - there is a reddit thread which also explains how the function used can be used to hash variable length things such as a [stream](https://www.reddit.com/r/crypto/comments/7ooot2/using_libsodiums_generic_hash_to_hash_a_file/)
- the notation of $\{0,1\}^\tau$ means a bit string of length $\tau$
    - From this [paper on hashing into Elliptic Curves](https://tools.ietf.org/html/draft-irtf-cfrg-hash-to-curve-01#page-4), they describe the following notation:
        - "bitstring of arbitrary length is denoted as $\{0, 1\}^*$"
- octet means base-256, and not to be confused with octal which is base-8

## 1. Hashing password into elliptic curve
### 5.2.3.  Simplified SWU Method

Given curve equation g(x) = x^3 + Ax + B, this algorithm works as follows:
   1. t = `HashToBase(\alpha)`
   2. $\alpha = \frac{-b}{a} * (1+\frac{1}{t^4 + t^2})$
   3. $\beta = -t^2 * \alpha$
   4. If $g(\alpha)$ is square, output $(\alpha, \sqrt{g(\alpha)})$
   5. Output $(\beta, \sqrt{g(\beta)})$

The following procedure implements this algorithm.  It outputs a point with affine coordinates.  It requires knowledge of A and B, the constants from the curve Weierstrass form.

Here is a definition of the function:
HashToBase(x): $H(x)[0:log_2(p) + 1]$, i.e., hash-truncate-reduce, where H is a cryptographic hash function, such as SHA256, and $p$ is the prime order of base field $F_p$.
Here is some psuedo code:
```
HashToBase(x, i)

Parameters:

 H - cryptographic hash function to use
 hbits - number of bits output by H
 p - order of the base field Fp
 label - context label for domain separation

Preconditions:

 floor(log2(p)) + 1 >= hbits

Input:

 x - value to be hashed, an octet string
 i - hash call index, a non-negative integer

Output:

 y - a value in the field Fp

Steps:

 1. t1 = H("h2c" || label || I2OSP(i, 4) || x)
 2. t2 = OS2IP(t1)
 3. y = t2 (mod p)
 4. Output y

where I2OSP, OS2IP [RFC8017] are used to convert an octet string to
and from a non-negative integer, and a || b denotes concatenation of
a and b.
```


In [51]:
from binascii import hexlify, unhexlify
from hashlib import sha1, sha256, sha384, sha512
import hashlib

# from https://tools.ietf.org/html/rfc8017#section-4.1

# Octet-String-to-Integer primitive
# def OS2IP(o_string):
#     # convert octet string to non-negative integer
#     return int(o_string, 256)

# https://stackoverflow.com/questions/39964383/implementation-of-i2osp-and-os2ip
def OS2IP(X):
        xLen = len(X)
        X = X[::-1]
        x = 0
        for i in range(xLen):
            x += X[i] * 256**i
        return x

# https://stackoverflow.com/questions/2267362/how-to-convert-an-integer-in-any-base-to-a-string
# def numberToBase(n, b):
#     if n == 0:
#         return [0]
#     digits = []
#     while n:
#         digits.append(int(n % b))
#         n //= b
#     return digits[::-1]

#Integer-to-Octet-String primitive
# def I2OSP(integer, length):
#     # find the length, or pass it in
#     if integer >= 256**length:
#         raise ValueError("integer {} too large".format(integer))
#     return numberToBase(integer, 256)[:length]

# https://stackoverflow.com/questions/39964383/implementation-of-i2osp-and-os2ip
# def I2OSP(x, xLen):
#         if x >= 256**xLen:
#             raise ValueError("integer too large")
#         digits = []

#         while x:
#             digits.append(int(x % 256))
#             x //= 256
#         for i in range(xLen - len(digits)):
#             digits.append(0)
#         return digits[::-1]

# https://en.wikipedia.org/wiki/Mask_generation_function
def I2OSP(integer: int, size: int = 4) -> str:
    return ''.join([chr((integer >> (8 * i)) & 0xFF) for i in reversed(range(size))])

print("OCT TEST: I2OSP(23) = {}, and then OS2IP(I2OSP(23)) = {}".format(I2OSP(23), OS2IP(I2OSP(23))))


def H(this):
    return sha256(this)

# def HashToBase(x, i):
    
    


TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [26]:
oct(1284341928424319814329834132908341034198743170431098714332140798143209874132970843127908341298704319708341901734907314901433198043123849129803419078413709841370984131947389738149780134938417094317809780143941738034190781983418934129823)

'0o14732070225451752437267220456106523475503242123547027145742667034746416523016631357560747177637576220351476354254034166072515470631160674163006412536226136652310363417610464331254603631027574307661730601371705154325032710667566452314272616703473053047122072237'